In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# import catheat
import openpyxl as xl
from openpyxl.utils.dataframe import dataframe_to_rows
import logging
import xlrd

In [2]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass

In [3]:
mastersheetCats = ['Tag',
                   'Cage',
                   'Ear',
                   'Sex', 
                   'Color',
                   'Name', 
                   'Genotype', 
                   'DOB',
                   'DOD',
                   'Father', 'Mother', 'Lineage','GenotypingLibrary','PlatePos', 'Notes']

In [4]:
def parseFrame(fn):
    df = pd.read_excel(fn,
                       dtype={'Column': str,
                              'Tag': str}, engine = "openpyxl").dropna(how='all') # drop empty colum
    # drop empty columns and typecast things that would otherwise be read as numbers
    if 'Strain' in df.columns: #not really necessary anymore
        df = df.rename(columns={'Strain': 'Lineage'})
    df.Column = df.Column.str.zfill(2) #convert column numbers to two character strings
    df.Lineage = df.Lineage.fillna("--") #fill in empty lineages so spaces don't get dropped
    df[['Father', 'Mother']] = df.Parents.str.lower().str.split("x", expand=True) #split parents on X or x
    df['TagClean'] = df.Tag.apply(lambda x: "t" + str(x)
                                  if is_number(x) else x) #convert tag numbers to strings with t appended
    df['PlatePos'] = [
        "{}-{}{}".format(plate, row, col)
        for plate, row, col in zip(df.Plate, df.Row, df.Column) #string formatting for master sheet
    ]
    df['Info'] = [
        '\n'.join([str(x), str(y)]) for x, y in zip(df['Tag'], df['Lineage'])
    ]
    locs = pd.read_pickle('96-wellLocs.pkl').sort_values(
        ['Column',
         'Row']).reset_index(drop=True)  # impt standard plate indexing
    df = locs.merge(df, how='outer') #merge into plate sheet so we have a full 96 well layout
    logging.log(msg = "Returned parsed sample sheet", level = 0)
    return df

In [5]:
# def mergePickleLocs(df):
#     locs = pd.read_pickle('96-wellLocs.pkl').sort_values(
#         ['Column',
#          'Row']).reset_index(drop=True)  # impt standard plate indexing
#     df = locs.merge(df, how='outer') #merge into plate sheet so we have a full 96 well layout
#     return df

In [6]:
def makeCleanLayout(fn, wb, df):
    sheetname = fn.rstrip('.xlsx') + '_CleanLayout'
    ws = wb.create_sheet(sheetname)    
    layout = df.pivot(index='Row', columns='Column',
                      values=['Info']).fillna("empty")
    rows = dataframe_to_rows(layout['Info'],
                             index=True,
                             header=True)
    for r in rows:
        ws.append(r)
    for cell in ws['A'] + ws[1]:
        cell.style = 'Pandas'
    logging.log(msg = "Returning clean layout", level = 0)
    return wb

In [7]:
def makeCleanSampleList(fn, wb, df):
    sheetname = fn.rstrip('.xlsx') + "_CleanSampleList"
    ws = wb.create_sheet(sheetname)
    cleanSamples = df[['Plate', 'Row', 'Column', 'TagClean', 'Lineage']]
    rows = dataframe_to_rows(cleanSamples,
                             index=False,
                             header=True)
    for r in rows:
        ws.append(r)
    return wb

In [8]:
def makeCleanMasterSheet(fn, wb, df, cats=mastersheetCats):
    sheetname = fn.rstrip('.xlsx') + "_Mastersheet"
    ws = wb.create_sheet(sheetname)
    df['GenotypingLibrary'] = ""
    master = df[cats].fillna("")
    rows = dataframe_to_rows(master,
                             index=False,
                             header=True)
    for r in rows:
        ws.append(r)
    return wb


In [9]:
def addHeaders(wb, operator = "KL"):
    for sheet in wb.sheetnames:
        wb[sheet].oddHeader.left.text = "&[Tab] \
                                            KL - &[Date]"
    return wb

In [10]:
fn = 'T1205271_21-11-15.xlsx'

In [11]:
# def process(fn, wb, df):
#     wb = makeCleanLayout(fn, wb, df)
#     wb = makeCleanSampleList(fn, wb, df)
#     wb = makeCleanMasterSheet(fn, wb, df)
#     wb = addHeaders(wb)
#     logging.log("Saving file")
#     wb.save(fn)
#     return wb

In [14]:
wb = xl.load_workbook(fn)
df = parseFrame(fn)
# fn = "testoutput.xlsx"

In [15]:
wb = makeCleanLayout(fn, wb, df)

In [16]:
# wb = makeCleanLayout(fn, wb, df)
wb = makeCleanSampleList(fn, wb, df)
wb = makeCleanMasterSheet(fn, wb, df)
wb = addHeaders(wb)

/home/kathleenleeper/.conda/envs/mouseMgmt/lib/python3.9/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [17]:
wb.save(fn)

In [ ]:
# for fn in fns:
#     wb = xl.load_workbook(fn)
#     df = parseFrame(fn)
#     wb = makeCleanLayout(fn, wb, df)
#     wb = makeCleanSampleList(fn, wb, df)
#     wb = makeCleanMasterSheet(fn, wb, df)
#     wb.save(fn)